In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

from pytorch_trainer import Trainer
from utils import get_loaders
from dataset import BadgerDataset

In [7]:
train = "train"
print('Number of signals in the train file', len(os.listdir(train)))

Number of signals in the train file 987


In [8]:
def accuracy_metric(predictions, labels):
    preds = torch.argmax(predictions, dim=1)
    correct_samples: float = torch.sum(preds == labels)
    total_samples: float = len(labels)
    
    return 100.0 * correct_samples / total_samples

In [19]:
def get_model(train_loader, val_loader):
    device = torch.device("cuda:0")
    
    number_of_classes: int = 2
    nn_model = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, padding=0),
                nn.BatchNorm2d(6),
                nn.MaxPool2d(kernel_size=2),
                nn.ReLU(inplace=True),
                nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, padding=0),
                nn.BatchNorm2d(16),
                nn.MaxPool2d(kernel_size=2),
                nn.ReLU(inplace=True),
                nn.Flatten(),
                nn.Linear(in_features=44944, out_features=120),
                nn.BatchNorm1d(120),
                nn.ReLU(inplace=True),
                nn.Linear(in_features=120, out_features=84),
                nn.BatchNorm1d(84),
                nn.ReLU(inplace=True),
                nn.Linear(in_features=84, out_features=number_of_classes),
                nn.Sigmoid()
          ).type(torch.cuda.FloatTensor)

    trainer = Trainer(nn_model, train_loader, val_loader, same_shape_tagret=False)
    loss_fn = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
    opt = optim.Adam(nn_model.parameters(), weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=15, gamma=0.5)

    trainer.train(opt=opt, 
                loss_fn=loss_fn, 
                metric_fn=accuracy_metric, 
                device=device, 
                epochs=5,
                scheduler=None)
    return nn_model

In [20]:
model_dataset = BadgerDataset(train_folder=train, transforms=transforms.Compose([
                           transforms.Resize((224, 224)),
                           transforms.ToTensor(),
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])                         
                       ]))

In [21]:
dataloaders, datasets = get_loaders(model_dataset=model_dataset)
model = get_model(train_loader=dataloaders["train"], val_loader=dataloaders["val"])

[ Training.. 1/5 ]


RuntimeError: The size of tensor a (4) must match the size of tensor b (3) at non-singleton dimension 0